In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [5]:
week13_link = 'http://www.footballlocks.com/nfl_odds_week_13.shtml#Closing%20NFL%20Odds%20Week%201,%202007'

#get html text
week13_html = requests.get(week13_link).text

soup = BeautifulSoup(week13_html, 'html.parser')

#want just table row data
tr = soup.find_all("tr")

len(tr)

#start of table
print(tr[51])

#last entry in table
print(tr[275])

<tr>
<td>11/30 8:25 ET</td>
<td>Washington</td>
<td>-1.5</td>
<td>At Dallas</td>
<td>47</td>
<td>-$125 +$105</td>
</tr>
<tr>
<td>12/4 8:30 ET</td>
<td>Carolina</td>
<td>-3</td>
<td>At Philadelphia</td>
<td>37.5</td>
<td>-$165 +$145</td>
</tr>


In [6]:
#want to 276 because it does not include last one
table = tr[51:276]

type(table[-1])

type(table)

string_table = [str(i) for i in table]

type(string_table[0])

week13_df = pd.DataFrame(string_table)

week13_df.columns = ['all_data']

week13_df.head()

,all_data
0,<tr>\n<td>11/30 8:25 ET</td>\n<td>Washington</...
1,<tr>\n<td>12/3 1:00 ET</td>\n<td>At Baltimore<...
2,<tr>\n<td>12/3 1:00 ET</td>\n<td>At Chicago</t...
3,<tr>\n<td>12/3 1:00 ET</td>\n<td>At Atlanta</t...
4,<tr>\n<td>12/3 1:00 ET</td>\n<td>New England</...


In [7]:
#want to clean the date data

dates = [x.split('td')[1] for x in week13_df['all_data']]

cleaned_dates = []
for x in dates:
    str1 = x.replace(">", "").replace("<", "")
    final_str = str1[:-1]
    cleaned_dates.append(final_str)

week13_df['date'] = cleaned_dates

In [8]:
#pull out favorites

week13_df['cleaned_favorites'] = week13_df['all_data'].map(lambda x: x.split('td')[3].replace('<', '').replace('>', '').replace('/', '')
                                                 if len(x.split('td'))>3 else np.nan)

#create underdog column

week13_df['cleaned_underdogs'] = week13_df['all_data'].map(lambda x: x.split('td')[7].replace('<', '').replace('>', '').replace('/', '')
                                                 if len(x.split('td'))>3 else np.nan)

#create spread column
week13_df['cleaned_spread'] = week13_df['all_data'].map(lambda x: x.split('td')[5].replace('<', '').replace('>', '').replace('/', '')
                                                 if len(x.split('td'))>3 else np.nan)

#create over_under column
week13_df['cleaned_over_under'] = week13_df['all_data'].map(lambda x: x.split('td')[9].replace('<', '').replace('>', '').replace('/', '')
                                                 if len(x.split('td'))>3 else np.nan)

#create money_odds column and clean data
money_odds = []
for x in week13_df['all_data']:
    try:
        money_odds.append(x.split('td')[11].replace('<', '').replace('>', '').replace('/', ''))
    except:
        money_odds.append(np.nan)
        
week13_df['money_odds'] = money_odds

#input week number for each game
week13_df['week_number'] = '13'

In [9]:
#want to input year for each game. on lines we want to decrease the count by one, len(week1_df.all_data[x].split('td')) = 3
year = 2018
year_list = []
for x in week13_df['all_data']:
    if len(x.split('td')) > 3:
        year_list.append(year)
    else:
        year_list.append(year)
        year -= 1

week13_df['year'] = year_list

In [10]:
#create a final dataframe for week 13
week13_final = week13_df[['date', 'year', 'week_number', 'cleaned_favorites', 'cleaned_underdogs', 'cleaned_spread', 
                        'cleaned_over_under', 'money_odds']]
week13_final.head()

,date,year,week_number,cleaned_favorites,cleaned_underdogs,cleaned_spread,cleaned_over_under,money_odds
0,11/30 8:25 ET,2018,13,Washington,At Dallas,-1.5,47,-$125 +$105
1,12/3 1:00 ET,2018,13,At Baltimore,Detroit,-2.5,43,-$140 +$120
2,12/3 1:00 ET,2018,13,At Chicago,San Francisco,-2.5,42,-$145 +$125
3,12/3 1:00 ET,2018,13,At Atlanta,Minnesota,-2,48.5,-$130 +$110
4,12/3 1:00 ET,2018,13,New England,At Buffalo,-8,49,-$390 +$320


In [11]:
#week13_final to pickle
week13_final.to_pickle('week13_final.pkl')